https://www.kaggle.com/competitions/share2022mlcomp01

get data

In [ ]:
! cd /content/drive/MyDrive/kaggle/

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/kaggle/"

In [ ]:
! kaggle competitions download -c share2022mlcomp01

 87% 40.0M/46.2M [00:00<00:00, 138MB/s]
100% 46.2M/46.2M [00:00<00:00, 125MB/s]


In [ ]:
! unzip share2022mlcomp01.zip

Archive:  share2022mlcomp01.zip
  inflating: sample_sabmission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

ytrain = train['label']
train = train.drop('id', axis=1)
train = train.drop('label', axis=1)
test = test.drop('id', axis=1)
#нормировать надо!!!!!!!!!!

In [ ]:
test

In [ ]:
print(sorted(ytrain.unique()))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [ ]:
from sklearn.impute import KNNImputer#, MissingIndicator
# KNNImputer потому что этот способ вставить значения
# куда лучше,чем просто какое либо среднее, но в то же время не слишком сложный
# parameters can be adjusted later (n_neighbors=3 for faster search)
# can be changed later

#ctrl+/ for fast comment
imputer = KNNImputer(n_neighbors=3)
tr = imputer.fit_transform(train)
train = pd.DataFrame(tr)
#флаги пропусков это хорошо(и корелляция пропусков)
#еще можно бы смотреть на данные(равномерность классов, корелляция,логистическая регрессия для ненужнвх фич?????, как то(кросс валидация; kfold) люди поняли, что первые 10к битые)

In [ ]:
imputer = KNNImputer(n_neighbors=3)
t = imputer.fit_transform(test)
test = pd.DataFrame(t)

In [ ]:
train.to_csv('trainNaN.csv', index = False)

In [ ]:
!	cp trainNaN.csv /content/drive/MyDrive/kaggle/

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train, ytrain, test_size=0.2)

данные подготовленны.Невозможно аугментировать из-за того, что признаки не несут функциональной нагрузки. Поиск алгоритма.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
#ут я пыталась искать сеткой, но долго крутилось P.S. тут я еще меняла параметр р для метрики
for w in ['uniform','distance']:
    for k in range(15,36,6):
        l = KNeighborsClassifier(n_neighbors=k, weights=w, p=2)#p=2(for metric), metric='minkowski'
        l.fit(X_train, y_train)
        y_pred = l.predict(X_val)
        print('w =', w, 'k =', k, 'acc =', accuracy_score(y_val, y_pred))

In [ ]:
X_train

In [ ]:
#мне подсказали про RandomSearch
model = KNeighborsClassifier()
params = { 'n_neighbors': [*range(15,36,6)], 'weights': ['uniform','distance'], 'metric': ['minkowski', 'cityblock', 'manhattan']}
search = RandomizedSearchCV(model, params, n_iter = 10)
s = search.fit(X_train, y_train)
s.best_params_

{'weights': 'distance', 'n_neighbors': 21, 'metric': 'minkowski'}

In [ ]:
s.best_score_

0.761375

In [ ]:
#тут я решила, что мало метрик передала и почему бы не посмотреть поближе, ну и почему бы чуть чуть(поиск все таки норм нашел) не поднять количество соседей(в первом пункте была динамика на чем больше тем лучше)
for m in ['minkowski', 'cosine', 'cityblock', 'euclidean', 'manhattan']:
    model = KNeighborsClassifier(n_neighbors = 27, weights = 'distance', metric = m)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    print('m=', m, 'acc =', accuracy_score(y_val, y_pred))

m= minkowski acc = 0.7586
m= cosine acc = 0.7603


In [ ]:
model = KNeighborsClassifier(n_neighbors = 27, weights = 'distance', metric = 'cosine')
model.fit(train, ytrain)
y = model.predict(test)

In [ ]:
y

array([1, 8, 9, ..., 5, 3, 3])

In [ ]:
d = {'id': [*range(len(y))], 'label': y}
df = pd.DataFrame(d)
df

In [ ]:
df.to_csv('Козырева_Елизавета.csv', index = False)